In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
import featurewiz

Imported featurewiz: advanced feature engg and selection library. Version=0.0.42
output = featurewiz(dataname, target, corr_limit=0.70,
                    verbose=2, sep=',', header=0, test_data='',
                    feature_engg='', category_encoders='')
Create new features via 'feature_engg' flag : ['interactions','groupby','target']
                                


In [ ]:
df_clean = pd.read_csv('/Users/williamjames/Desktop/CSCI/487/final_data.csv')
df_clean = df_clean.drop(columns=['Unnamed: 0'])
df_home = df_clean.loc[(df_clean['location']==2)]
X = df_home.drop(columns=['win','location'])
y = df_home['win']



In [8]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import cross_val_score

In [ ]:
k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    RF = RandomForestClassifier(n_estimators=25,n_jobs=-1,max_depth=13)
    sfs1 = sfs(RF, k_features=k, forward=True, verbose=2, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    RF.fit(X_train,y_train)
    accuracyCV = cross_val_score(clf, X[feat_names], y, scoring='accuracy', cv = 3)
    k_features.append(k)
    method.append('forward')
    features.append(feat_names)
    score.append(np.mean(accuracyCV))
forward = list(zip(k_features,direction,features,score))

In [ ]:
k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    RF = RandomForestClassifier(n_estimators=25,n_jobs=-1,max_depth=13)
    sfs1 = sfs(RF, k_features=k, forward=False, verbose=1, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    RF.fit(X_train,y_train)
    accuracyCV = cross_val_score(clf, X[feat_names], y, scoring='accuracy', cv = 3) #Use cross validation on 3 folds to determine accuracy

    k_features.append(k)
    direction.append('backwards')
    features.append(feat_names)
    score.append(np.mean(accuracyCV))

In [38]:
backwards = list(zip(k_features,direction,features,score))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
k_features = []
direction = []
scores = []
feats = []
accuracy = []
for i in range(5,19):
    rfe = RFE(estimator=RandomForestClassifier(n_estimators=100,n_jobs=-1,max_depth=13), n_features_to_select=i)
    rfe.fit(X,y)
    model = RandomForestClassifier(n_estimators=100,n_jobs=-1,max_depth=13)
    support = rfe.support_
    feature_names = X_train.columns
    rfe_feats = feature_names[support]
    # fit the model on all available data
    n_scores = cross_val_score(model, X[rfe_feats], y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(i)
    direction.append('rfe')
    scores.append(np.mean(n_scores))
    feats.append(rfe_feats)

rfe_res = list(zip(k_features,direction,feats,scores))

In [ ]:
from sklearn.decomposition import PCA

k_features = []
direction = []
scores = []
feats = []

for i in range(1,20):
    pca = PCA(n_components=i)
    X_pca = pca.fit_transform(X)
    # Get the transformed dataset
    X_pca = pd.DataFrame(X_pca)
    clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,max_depth=13)
    n_scores = cross_val_score(clf, X_pca, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(i)
    method.append('pca')
    scores.append(np.mean(n_scores))
    feats.append('all')

In [48]:
pca_data = list(zip(k_features,direction,feats,scores))

In [49]:
bwd = pd.DataFrame(data=backwards[:],columns=['k_features','direction','features','score'])
fwd = pd.DataFrame(data=forward[:],columns=['k_features','direction','features','score'])
rfe_results = pd.DataFrame(data=rfe_res[:],columns=['k_features','direction','features','score'])
pca_results = pd.DataFrame(data=pca_data[:],columns=['k_features','direction','features','score'])
results = pd.concat([bwd,fwd,rfe_results,pca_results])


In [51]:
results.sort_values('score',ascending=False)

,k_features,direction,features,score
3,8,backwards,"[G_PTS, C_PTS, starter_MIN, EFG, FT_rate, DREB...",0.672654
10,15,rfe,"Index(['HHI', 'HHI_assists', 'G_PTS', 'starter...",0.669511
11,16,rfe,"Index(['HHI', 'HHI_assists', 'G_PTS', 'starter...",0.669511
10,11,pca,all,0.668612
8,9,pca,all,0.667714
...,...,...,...,...
14,19,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.643916
7,12,forwards,"[HHI, F_PTS, C_PTS, starter_PTS, EFG, FT_rate,...",0.643467
2,7,backwards,"[F_PTS, TO_rate, EFG, OREB_rate, DREB_rate, op...",0.642119
3,8,rfe,"Index(['TO_rate', 'EFG', 'FT_rate', 'OREB_rate...",0.636731


In [6]:
from sklearn.model_selection import cross_val_score

fts = []
direction = []
k_features = []
accuracy = []
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,max_depth=13)
feats = ['opp_play_percent', 'play_percent', 'TO_rate', 'OREB_rate', 'FT_rate', 'opp_TO_rate', 'opp_OREB_rate', 'starter_MIN', 'C_PTS', 'F_PTS', 'opp_FT_rate']
X = df_home[feats]
y = df_home['win']
n_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
fts.append(feats)
k_features.append(len(feats))
direction.append('featurewiz')
accuracy.append(np.mean(n_scores))
fwiz = list(zip(k_features,direction,fts,accuracy))
fw_res = pd.DataFrame(data=fwiz[:],columns=['k_features','direction','features','score'])

0.6636731028289178


In [9]:
results = pd.read_csv('Random_Forest_Results.csv')
results = pd.concat([results,fw_res])
results
results.to_csv('Random_Forest_Results.csv',index=False)

In [52]:
results.to_csv('Random_Forest_Results.csv',index=False)